In [ ]:
import json
from fenics import *
import numpy as np
import logging
import mshr

#Turn off logging
logging.getLogger("FFC").setLevel(logging.ERROR)
logging.getLogger("UFL_LEGACY").setLevel(logging.ERROR)
logging.getLogger("UFL").setLevel(logging.ERROR)
def read_dictionary(fname):
    with open(fname) as json_file:
        pars = json.load(json_file)
    return pars

def read_state(fname,FE):

    data = read_dictionary(fname + '.json')
    it = data['step']
    t  = data['time']

    mesh = Mesh()

    f=HDF5File(mesh.mpi_comm(),fname+'.h5', 'r')
    f.read(mesh,"mesh",False)
    V = FunctionSpace(mesh,FE)
    q = Function(V)
    f.read(q,"q")
    f.close()
    return mesh,q,it,t

def read_mesh(fname):

    data = read_dictionary(fname + '.json')
    it = data['step']
    t  = data['time']

    mesh = Mesh()

    f=HDF5File(mesh.mpi_comm(),fname+'.h5', 'r')
    f.read(mesh,"mesh",False)
    f.close()
    return mesh

#mesh = read_mesh('partest/kiln3D/phi0')
#f = File('meshtest.pvd')
#f << mesh

FE_ac = FiniteElement('CG', tetrahedron, 1)

# Space for Stokes problem
FE_u = VectorElement('CG', tetrahedron, 2)
FE_p = FiniteElement('CG', tetrahedron, 1)
FEs = MixedElement([FE_u, FE_p])
f1 = File('../kiln3D_tilted/kiln3D_tilted/phitest.pvd')
f2 = File('../kiln3D_tilted/kiln3D_tilted/vtest.pvd')
for i in range(0,97,5):
    _,phi,it,t = read_state('../kiln3D_tilted/kiln3D_tilted/phi'+str(i),FE_ac)
    _,vp,it,t = read_state('../kiln3D_tilted/kiln3D_tilted/v'+str(i),FEs)
    phi.rename('phi','phi')
    f1 << (phi,t)
    v,p = vp.split()
    v.rename('v','v')
    f2 << (v,t)
    
    print(it,t)

In [ ]:
print("Number of DoFs:", len(v.vector()))
print("Number of DoFs:", len(phi.vector()))